# Homework #1 (Due 09/18/2019, 11:59pm)
## Maximum Likelihood Learning and Bayesian Inference

**AM 207: Advanced Scientific Computing**<br>
**Instructor: Weiwei Pan**<br>
**Fall 2019**

**Name: Dimitris Vamvourellis**

**Students collaborators:**

### Instructions:

**Submission Format:** Use this notebook as a template to complete your homework. Please intersperse text blocks (using Markdown cells) amongst `python` code and results -- format your submission for maximum readability. Your assignments will be graded for correctness as well as clarity of exposition and presentation -- a “right” answer by itself without an explanation or is presented with a difficult to follow format will receive no credit.

**Code Check:** Before submitting, you must do a "Restart and Run All" under "Kernel" in the Jupyter or colab menu. Portions of your submission that contains syntactic or run-time errors will not be graded.

**Libraries and packages:** Unless a problems specifically asks you to implement from scratch, you are welcomed to use any `python` library package in the standard Anaconda distribution.

In [1]:
### Import basic libraries
import numpy as np
import math as ma
import pandas as pd
import sklearn as sk
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
%matplotlib inline

## Problem Description
In the competitive rubber chicken retail market, the success of a company is built on satisfying the exacting standards of a consumer base with refined and discriminating taste. In particular, customer product reviews are all important. But how should we judge the quality of a product based on customer reviews?

On Amazon, the first customer review statistic displayed for a product is the ***average rating***. The following are the main product pages for two competing rubber chicken products, manufactured by Lotus World and Toysmith respectively:


Lotus World |  Toysmith
- |  - 
![alt](lotus1.png) |  ![alt](toysmith1.png)

Clicking on the 'customer review' link on the product pages takes us to a detailed break-down of the reviews. In particular, we can now see the number of times a product is rated a given rating (between 1 and 5 stars).

Lotus World |  Toysmith
- |  - 
![alt](lotus2.png) |  ![alt](toysmith2.png)


In the following, we will ask you to build statistical models to compare these two products using the observed rating. Larger versions of the images are available in the data set accompanying this notebook.



## Part I: A Maximum Likelihood Model
1. **(Model Building)** Suppose that for each product, we can model the probability of the value each new rating as the following vector:
$$
\theta = [\theta_1, \theta_2, \theta_3, \theta_4, \theta_5]
$$
  where $\theta_i$ is the probability that a given customer will give the product $i$ number of stars. That is, each new rating (a value between 1 and 5) has a categorical distribution $Cat(\theta)$. Represent the observed ratings of an Amazon product as a vector $R = [r_1, r_2, r_3, r_4, r_5]$ where, for example, $r_4$ is the number of $4$-star reviews out of a total of $N$ ratings. Write down the likelihood of $R$. That is, what is $p(R| \theta)$?

  **Note:** The observed ratings for each product should be read off the image files included in the dataset.

**ANSWER** 

The likelihood of R given N observations can be written using the pmf of the Multinomial distribution as follows:

$$p(R| \theta) = \frac{N!}{r_1!r_2!r_3!r_4!r_5!}\theta_1^{r_1}\theta_2^{r_2}\theta_3^{r_3}\theta_4^{r_4}\theta_5^{r_5}$$

Hence, the log-likelihood is given by
$$\ell(\theta) = \log{p(R| \theta)} = \log(N!)-\displaystyle\sum_{i=1}^{5}{\log(r_i!)}+ \displaystyle\sum_{i=1}^{5}{r_i \log(\theta_i)} $$


2. **(Model Fitting)** Find the maximum likelihood estimator of $\theta$ for the Lotus World model; find the MLE of $\theta$ for the Toysmith model. You need to make a reasonably mathematical argument for why your estimate actually maximizes the likelihood (i.e. recall the criteria for a point to be a global optima of a function).

  *Note:* I recommend deriving the MLE using the general expression of the likelihood. That is, derive the posterior using the variable $R$, then afterwards plug in your specific values of $R$ for each product.

**ANSWER**

To derive the MLE of the $\theta$ vector which maximizes the likelihood $p(R| \theta)$, we need to solve the following constrained optimization problem:
$$
\mathrm{max}\;\ell(\theta),\quad\displaystyle\sum_{i=1}^{5}{\theta_i}=1
$$
whose Lagrangian is given by:
$$
J(\theta, \lambda) = \ell(\theta) - \lambda(\displaystyle\sum_{i=1}^{5}{\theta_i} - 1).
$$

The gradient of the Lagrangian $J$ with respect to $(\theta, \lambda)$ is the vector $\nabla_{(\theta, \lambda)} J(\theta,\lambda) = \left[\frac{\partial J}{\partial \theta_1}, \frac{\partial J}{\partial \theta_2}, \frac{\partial J}{\partial \theta_3}, \frac{\partial J}{\partial \theta_4}, \frac{\partial J}{\partial \theta_5}, \frac{\partial J}{\partial \lambda} \right]$, where the partial derivatives are given by:
\begin{aligned}
\frac{\partial J}{\partial \theta_1} &= \frac{r_1}{\theta_1} - \lambda\\
\frac{\partial J}{\partial \theta_2} &= \frac{r_2}{\theta_2} - \lambda\\
\frac{\partial J}{\partial \theta_3} &= \frac{r_3}{\theta_3} - \lambda\\
\frac{\partial J}{\partial \theta_1} &= \frac{r_4}{\theta_4} - \lambda\\
\frac{\partial J}{\partial \theta_1} &= \frac{r_5}{\theta_5} - \lambda\\
\frac{\partial J}{\partial \lambda} &= \theta_1 + \theta_2 + \theta_3 + \theta_4 + \theta_5 - 1
\end{aligned}

 
The stationary points of the Lagrangian can be obtained by solving the following system of equations:

$$
\begin{cases}
\frac{\partial J}{\partial \theta_1} &= \frac{r_1}{\theta_1} - \lambda= 0\\
\frac{\partial J}{\partial \theta_2} &= \frac{r_2}{\theta_2} - \lambda= 0\\
\frac{\partial J}{\partial \theta_3} &= \frac{r_3}{\theta_3} - \lambda= 0\\
\frac{\partial J}{\partial \theta_1} &= \frac{r_4}{\theta_4} - \lambda= 0\\
\frac{\partial J}{\partial \theta_1} &= \frac{r_5}{\theta_5} - \lambda= 0\\
\frac{\partial J}{\partial \lambda} &= \theta_1 + \theta_2 + \theta_3 + \theta_4 + \theta_5 - 1= 0
\end{cases}
$$


Solving this system, we get a unique solution at:

$$
\begin{cases}
\theta_1 = \frac{r_1}{\lambda}&\\
\theta_2 = \frac{r_2}{\lambda}&\\
\theta_3 = \frac{r_3}{\lambda}&\\
\theta_4 = \frac{r_4}{\lambda}&\\
\theta_5 = \frac{r_5}{\lambda}&\\
\theta_1 + \theta_2 + \theta_3 + \theta_4 +\theta_5  = 1&
\end{cases}
$$

where, $\lambda=r_1+r_2+r_3+r_4+r_5=N$.

**Characterize Stationary Point**

We have a unique stationary point but we need to check whether this is a maximum or minimum of the log likelihood. Testing the value of the log-likelihood at the stationary point of the Lagrangian (where $\theta_i = \frac{r_i}{N}$), and another point on the line $\theta_1 + \theta_2 + \theta_3 + \theta_4 + \theta_5  = 1$, we can check whether this is a minimum or maximum. If the likelihood is greater at the stationary point, then this is a maximum. Then, since there is only one optimum, it has to be a global optimum. For example, we can do this check by calculating the likelihood using LW actual ratings as follows.

Given the above result, we know that the MLE of $\theta$ is given by the vector which holds the proportions of each rating. Therefore, for the Lotus World model, the MLE of $\theta$ is given by
$$\theta_{MLE_{LW}} = [0.06, 0.04, 0.06, 0.17, 0.67].$$

Respectively, for the Toysmith model, the MLE of $\theta$ is given by
$$\theta_{MLE_{Toysmith}} = [0.14, 0.08, 0.07, 0.11, 0.60].$$

We will plug $\theta_{MLE_{LW}}$ in the log-likelihood and compare its value with the log-likelihood when plugging another arbitrary point $\theta^{\prime} = [0.2, 0.2, 0.2, 0.2, 0.2].$ The only term of the log likelihood that depends on $\theta$ is $\displaystyle\sum_{i=1}^{5}{r_i \log(\theta_i)}$. Hence we can do the comparisons based on this term.

For this, we will need the approximate absolute counts of ratings per number of stars given, which we will calculate below for LW.

In [14]:
print("Lotus World: 5 stars=%f, 4 stars=%f, 3 stars=%f, 2 stars=%f  1 star=%f" 
     % (round(0.667*162),round(0.17*162),round(0.06*162),round(0.04*162),round(0.06*162)))

Lotus World: 5 stars=108.000000, 4 stars=28.000000, 3 stars=10.000000, 2 stars=6.000000  1 star=10.000000


In [20]:
R_test = np.array([10, 6, 10, 28, 108])
theta_MLE = np.array([0.06, 0.04, 0.06, 0.17, 0.67])
theta_prime = np.array([0.2, 0.2, 0.2, 0.2, 0.2])

sum_mle = np.dot(np.log(theta_MLE), R_test)
sum_prime = np.dot(np.log(theta_prime), R_test)
print(sum_mle, sum_prime)

-168.44783805099195 -260.72894181432423


As we can see, log-likelihood is greater at $\theta_{MLE_{LW}}$, hence the log-likelihood is maximized and this is a global maximum along the line $\theta_1 + \theta_2 + \theta_3 + \theta_4 + \theta_5  = 1$ at the stationary point.

3. **(Model Interpretation)** Based on your MLE of $\theta$'s for both models, do you feel confident deciding if one product is superior to another? Why or why not?

We don't feel confident deciding if one product is superior to the other since there is no striking difference in proportions of ratings for each category of stars. Lotus World product has a slightly larger proportion of 5s and a slightly lower proportion of 1s but this small difference is not enough to claim that one product is superior to the other. The number of ratings that we have for Lotus World is not large enough and significantly lower than the number of ratings for Toysmith. As it is known, the MLE tends to overfit when there is scarcity of data, hence we are not confident that this estimate is close to the true parameter $\theta$. 

## Part II: A Bayesian Model

1. **(Model Building)** Suppose you are told that customer opinions are very polarized in the retail world of rubber chickens, that is, most reviews will be 5 stars or 1 stars (with little middle ground). What would be an appropriate $\alpha$ for the Dirichlet prior on $\theta$? Recall that the Dirichlet pdf is given by:
$$
p_{\Theta}(\theta) = \frac{1}{B(\alpha)} \prod_{i=1}^k \theta_i^{\alpha_i - 1}, \quad B(\alpha) = \frac{\prod_{i=1}^k\Gamma(\alpha_i)}{\Gamma\left(\sum_{i=1}^k\alpha_i\right)},
$$
where $\theta_i \in (0, 1)$ and $\sum_{i=1}^k \theta_i = 1$, $\alpha_i > 0 $ for $i = 1, \ldots, k$.

**ANSWER**

If our prior belief is that most reviews are either 5 or 1 stars, then we can set a Dirichlet prior as follows:

$$\alpha = [5, 1, 1, 1, 5].$$

In this way, we specify our belief that it is equally probable to have 1 star review or a 5 star review, but less mass is allocated in the middle.

2. **(Inference)** Analytically derive the posterior distribution (using the likelihoods you derived in Part I) for each product.

  *Note:* I recommend deriving the posterior using the general expression of a Dirichelet pdf. That is, derive the posterior using the variable $\alpha$, then afterwards plug in your specific values of $\alpha$ when you need to.

**ANSWER**

The posterior distribution is given by

$$p(\theta | R) = \frac{p(R | \theta)p(\theta)}{p(R)} = \frac{N!\frac{1}{B(\alpha)}\prod_{i=1}^5\frac{\theta_i^{r_i}}{r_i!}\prod_{i=1}^5 \theta_i^{\alpha_i - 1}}{p(R)}$$

We can bundle together all the constants and rewrite the posterior as

$$p(\theta | R) = const * \prod_{i=1}^5 \theta_i^{\alpha_i+r_i - 1}$$

where $const = \frac{N!\prod_{i=1}^5\frac{1}{r_i!}}{B(\alpha)p(R)}$. We recognize that the posterior is a Dirichlet distribution with parameters given by the vector $R+\alpha$ and the constant term is the normalization constant. Hence, the posterior is given by

$$p(\theta | R) = \frac{1}{B(\alpha + R)} \prod_{i=1}^5 \theta_i^{\alpha_i+r_i - 1}.$$

To calculate the posterior distributions for each model, we will need the approximate absolute counts of ratings per number of stars given, which we will calculate below for each toy.

In [57]:
print("Lotus World: 5 stars=%f, 4 stars=%f, 3 stars=%f, 2 stars=%f  1 star=%f" 
     % (round(0.667*162),round(0.17*162),round(0.06*162),round(0.04*162),round(0.06*162)))
print("Toysmith: 5 stars=%f, 4 stars=%f, 3 stars=%f, 2 stars=%f  1 star=%f" 
     % (round(0.6*410),round(0.11*410),round(0.07*410),round(0.08*410),round(0.14*410)))
      

Lotus World: 5 stars=108.000000, 4 stars=28.000000, 3 stars=10.000000, 2 stars=6.000000  1 star=10.000000
Toysmith: 5 stars=246.000000, 4 stars=45.000000, 3 stars=29.000000, 2 stars=33.000000  1 star=57.000000


Hence, for a prior Dirichlet disribution with $\alpha = [5, 1, 1, 1, 5]$ and using the absolute counts we calculated above, the posterior distribution for the Lotus World model is given by

$$p(\theta_{LW} | R_{LW}) = \frac{1}{B(\alpha+R_{LW})}\theta_1^{10+5-1}\theta_2^{6+1-1}\theta_3^{10+1-1}\theta_4^{28+1-1}\theta_5^{108+5-1}=\frac{1}{B(\alpha+R_{LW})}\theta_1^{14}\theta_2^{6}\theta_3^{10}\theta_4^{28}\theta_5^{112}$$, where $R_{LW} = [10, 6, 10, 28, 108]$.

Respectively, for the same prior, the posterior distribution for the Toysmith model is given by

$$p(\theta_{Toysmith} | R_{Toysmith}) = \frac{1}{B(\alpha+R_{Toysmith})}\theta_1^{61}\theta_2^{33}\theta_3^{29}\theta_4^{45}\theta_5^{250}$$, where $R_{Toysmith} = [246, 45, 29, 33, 57]$.



3. **(The Maximum A Posterior Estimate)** Analytically or empirically compute the MAP estimate of $\theta$ for each product, using the $\alpha$'s you chose in Problem 1. How do these estimates compare with the MLE? Just for this problem, compute the MAP estimate of $\theta$ for each product using a Dirichelet prior with hyperparameters $\alpha = [1, 1, 1, 1, 1]$. Make a conjecture about the effect of the prior on the difference between the MAP estimates and the MLE's of $\theta$.

**ANSWER**

Since the posterior is a Dirichlet distribution, we have a closed formula for its mode (i.e. the MAP estimate of $\theta$). Particularly, the mode for each $\theta_i$ in $\theta$ drawn from a Dirichlet($\alpha+R$) is given by

$$\theta_{i_{MAP}} = \frac{\alpha_i+r_i-1}{\displaystyle\sum_{i=1}^{5}{(\alpha_i+r_i-1)}}$$

Substituring the values of $\alpha$ and $R$ for each product, we get the corresponding MAP estimate of $\theta$ as shown below.

In [61]:
#calculating MAP estimated for uneven prior
R_LW = np.array([10, 6, 10, 28, 108])
R_toy = np.array([57, 33, 29, 45, 246])
a_prior = np.array([5, 1, 1, 1, 5])
theta_MAP_LW = (R_LW + a_prior - 1)/(R_LW + a_prior - 1).sum()
theta_MAP_toy = (R_toy + a_prior - 1)/(R_toy + a_prior - 1).sum()
print("For LW, theta_MAP:", theta_MAP_LW)
print("For Toysmith, theta_MAP:", theta_MAP_toy)

For LW, theta_MAP: [0.08235294 0.03529412 0.05882353 0.16470588 0.65882353]
For Toysmith, theta_MAP: [0.14593301 0.07894737 0.06937799 0.1076555  0.59808612]


Hence the MAP estimate for Lotus World and Toysmith respectively is 

$$\theta_{MAP_{LW}} = [0.082, 0.035, 0.059, 0.165, 0.659],$$
$$\theta_{MAP_{Toysmith}} = [0.146, 0.079, 0.069, 0.108, 0.598],$$

whereas the MLE estimates found above are

$$\theta_{MLE_{LW}} = [0.06, 0.04, 0.06, 0.17, 0.67].$$
$$\theta_{MLE_{Toysmith}} = [0.14, 0.08, 0.07, 0.11, 0.60].$$

As it is made apparent, there is no big difference between the MLE and the MAP estimates for $\theta$ in each case. The biggest difference can be observed in $\theta_1$ which is slightly larger under the MAP estimate and in $\theta_5$ which is larger under the MLE for the LW product.

In [3]:
#calculating MAP estimates for flat prior
R_LW = np.array([10, 6, 10, 28, 108])
R_toy = np.array([57, 33, 29, 45, 246])
a_prior_flat = np.array([1, 1, 1, 1, 1])
theta_MAP_LW = (R_LW + a_prior_flat - 1)/(R_LW + a_prior_flat - 1).sum()
theta_MAP_toy = (R_toy + a_prior_flat - 1)/(R_toy + a_prior_flat - 1).sum()
print("For LW, theta_MAP:", theta_MAP_LW)
print("For Toysmith, theta_MAP:", theta_MAP_toy)

For LW, theta_MAP: [0.0617284  0.03703704 0.0617284  0.17283951 0.66666667]
For Toysmith, theta_MAP: [0.13902439 0.0804878  0.07073171 0.1097561  0.6       ]


Under a flat prior, the MAP estimates are

$$\theta_{MAP_{LW}} = [0.062, 0.037, 0.062, 0.173, 0.667],$$
$$\theta_{MAP_{Toysmith}} = [0.139, 0.080, 0.071, 0.110, 0.6].$$

In this case, the MAP estimates are almost equal to the MLE estimates. By using a flat prior, we demonstrate our belief that we don't have any prior knowledge about the distribution of the proportion of ratings (i.e. the symmetric Dirichlet distribution is equivalent to a uniform distribution in higher dimensions). In other words, this is an uninformative prior and for this reason the posterior is dominated by the likelihood after seeing many examples. As a result, the MAP and the MLE estimates almost coincide if we set a completely flat prior.

4. **(The Posterior Mean Estimate)** Analytically or empirically compute the posterior mean estimate of $\theta$ for each product, using the $\alpha$'s you chose in Problem 1. How do these estimates compare with the MAP estimates and the MLE?

**ANSWER**

Again, since the posterior is a Dirichlet$(R+\alpha)$ distribution, we have a formula for the mean of each $\theta_i$ in $\theta$ given by

$$\theta_{i_{mean}} = \frac{\alpha_i+r_i}{\displaystyle\sum_{i=1}^{5}{(\alpha_i+r_i)}}$$

Substituring the values of $\alpha$ and $R$ for each product, we get the corresponding posterior mean estimate of $\theta$ as shown below. 

In [4]:
a_prior = np.array([5, 1, 1, 1, 5])
theta_mean_LW = (R_LW + a_prior)/(R_LW + a_prior).sum()
theta_mean_toy = (R_toy + a_prior)/(R_toy + a_prior).sum()
print("For LW, theta_mean:", theta_mean_LW)
print("For Toysmith, theta_mean:", theta_mean_toy)

For LW, theta_mean: [0.08571429 0.04       0.06285714 0.16571429 0.64571429]
For Toysmith, theta_mean: [0.1465721  0.08037825 0.07092199 0.10874704 0.59338061]


We can validate the above result empirically by drawing samples from each posterior and calculating the sample mean.

In [6]:
samples_LW = np.random.dirichlet(tuple(a_prior+R_LW), 10000)
samples_toy = np.random.dirichlet(tuple(a_prior+R_toy), 10000)
print("For LW, posterior mean estimate is",np.mean(samples_LW, axis=0))
print("For Toysmith, posterior mean estimate is",np.mean(samples_toy, axis=0))

For LW, posterior mean estimate is [0.08581411 0.0398802  0.06279848 0.16564791 0.6458593 ]
For Toysmith, posterior mean estimate is [0.14658839 0.0803993  0.07096977 0.10879263 0.59324991]


5. **(The Posterior Predictive Estimate)** Sample 1000 rating vectors from the posterior predictive for each product, using the $\alpha$'s you chose in Problem 1. Use the average of the posterior predictive samples to estimate $\theta$. How do these estimates compare with the MAP, MLE, posterior mean estimate of $\theta$?

**ANSWER**

In [117]:
#function to draw samples from the posterior predictive distribution
def get_post_pred_samples(prior_a, N, R, samples):
    post_pred_samples = np.zeros((samples, prior_a.shape[0]))
    #draw a sample from the posterior and use it to draw a sample from the multinomial predictive.
    for i in range(0,samples):
        theta_sample = np.random.dirichlet(tuple(a_prior+R), 1)[0]
        post_pred_samples[i] = np.random.multinomial(N, theta_sample)
    return post_pred_samples

In [118]:
post_pred_LW = get_post_pred_samples(a_prior, 162, R_LW, 1000)
post_pred_toy = get_post_pred_samples(a_prior, 410, R_toy, 1000)

In [119]:
print("The mean of the posterior predictive distribution for LW is", np.mean(post_pred_LW, axis=0))


The mean of the posterior predictive distribution for LW is [ 13.697   6.671  10.183  27.089 104.36 ]


In [120]:
print("The mean of the posterior predictive distribution for Toysmith is", np.mean(post_pred_toy, axis=0))

The mean of the posterior predictive distribution for Toysmith is [ 59.775  33.404  29.144  44.093 243.584]


We can divide the mean vectors with the sum of ratings to get another estimate for $\theta$ as follows.

In [121]:
theta_pred_LW = np.mean(post_pred_LW, axis=0)/162.0
theta_pred_toy = np.mean(post_pred_toy, axis=0)/410.0

print("For LW, the estimate of theta based on the mean of the posterior predictive is", theta_pred_LW)
print("For Toysmith, the estimate of theta based on the mean of the posterior predictive is", theta_pred_toy)

For LW, the estimate of theta based on the mean of the posterior predictive is [0.08454938 0.04117901 0.06285802 0.16721605 0.64419753]
For Toysmith, the estimate of theta based on the mean of the posterior predictive is [0.14579268 0.08147317 0.07108293 0.1075439  0.59410732]


Hence, the estimate $\theta$ for each product respectively given the mean of the posterior predictive distribution is

$$\theta_{pred_{LW}} = [0.085, 0.041, 0.063, 0.167, 0.644],$$
$$\theta_{pred_{Toysmith}} = [0.146, 0.080, 0.071, 0.108, 0.594].$$

For comparison purposes, we attach the results from the other methods below.

$$\theta_{mean_{LW}} = [0.086, 0.04, 0.063, 0.166, 0.646],$$
$$\theta_{mean_{Toysmith}} = [0.146, 0.080, 0.070, 0.109, 0.593],$$

$$\theta_{MAP_{LW}} = [0.082, 0.035, 0.059, 0.165, 0.659],$$
$$\theta_{MAP_{Toysmith}} = [0.146, 0.079, 0.069, 0.108, 0.598],$$

$$\theta_{MLE_{LW}} = [0.06, 0.04, 0.06, 0.17, 0.67],$$
$$\theta_{MLE_{Toysmith}} = [0.14, 0.08, 0.07, 0.11, 0.60].$$

Again, these estimates are very close to the MAP and posterior mean estimates while there is a noticeable difference with the MLE regarding $\theta_1$ and $\theta_5$ especially. Also, although the difference with MAP is small, $\theta_{pred}$ estimates are a bit closer to the mean posterior ones.

6. **(Model Evaluation)** Compute the 95% credible interval of $\theta$ for each product (*Hint: compute the 95% credible interval for each $\theta_i$, $i=1, \ldots, 5$*). For which product is the posterior mean and MAP estimate more reliable and why? 

In [124]:
samples_LW = np.random.dirichlet(tuple(a_prior+R_LW), 10000)
samples_toy = np.random.dirichlet(tuple(a_prior+R_toy), 10000)

In [130]:
print("For LW, the 2.5th percentile is:", np.percentile(samples_LW, 2.5, axis=0))
print("For LW, the 97.5th percentile is:", np.percentile(samples_LW, 97.5, axis=0))

For LW, the 2.5th percentile is: [0.04861892 0.01617297 0.03166043 0.11465613 0.57428849]
For LW, the 97.5th percentile is: [0.13136833 0.07303282 0.10305547 0.22394475 0.71511221]


Hence, the 95% confidence interval for each entry in $\theta_{LW}$ is given by

$$[(0.049,0.131),(0.016,0.073),(0.031,0.103),(0.115,0.224),(0.574,0.715)]$$

In [131]:
print("For Toysmith, the 2.5th percentile is:", np.percentile(samples_toy, 2.5, axis=0))
print("For Toysmith, the 97.5th percentile is:", np.percentile(samples_toy, 97.5, axis=0))

For Toysmith, the 2.5th percentile is: [0.11468113 0.05611783 0.04874188 0.08095351 0.54639157]
For Toysmith, the 97.5th percentile is: [0.18121776 0.10714582 0.09712603 0.14043751 0.64037328]


Hence, the 95% confidence interval for each entry in $\theta_{Toysmith}$ is given by

$$[(0.115,0.181),(0.056,0.107),(0.049,0.097),(0.081,0.140),(0.546,0.640)]$$

As we can see from the 95% confidence intervals provided above, for most of the parameters and especially for $\theta_1$ and $\theta_5$, the confidence intervals for Toysmith are tighter. This is expected since we had significantly more observations for Toysmith, hence the posterior distribution is more peaked at the MAP. For this reason, the posterior mean or the MAP estimate is more reliable for Toysmith, since the uncertainty around them is smaller compared to the Lotus World product (as it is reflected in the corresponding 95% intervals).

## Part III: Comparison
1. **(Summarizing Customer Ratings)** Recall that on Amazon, the first customer review statistic displayed for a product is the average rating. Name at least one problem with ranking products based on the average customer rating.

**ANSWER**

The average rating can be very misleading and inconsistent when the total number of ratings is small. In an extreme scenario, if we had only one rating of 5 stars, the average would be 5 stars. If one more rating of 1 stars was added, the average would change to 3 which gives a totally different sentiment to the buyer. In other words, the average rating does not depict the total number of reviews taken into account and in turn when seeing it in isolation it does not give any sense of the uncertainty around it. For example, let product A have an average rating of 5 stars from 1 review and product B have an average rating of 4.5 stars from 100 reviews. Checking the average rating in isolation would lead us to believe that product A is better. However, in reality, the uncertainty about the true average rating of product A is much greater than the uncertainty for product B. 

2. **(Comparison of Point Estimates)** Which point estimate (MAP, MLE, posterior mean or posterior predictive estimate) of $\theta$, if any, would you feel choose to rank the two Amazon products? Why? 

  *Hint: think about which of these estimates are equivalent (if any). If they are not equivalent, what are the special properties of each estimate? What aspect of the data or the model is each estimate good at capturing?*
  
   **Note:** we're not looking for "the correct answer" here. We are looking for a sound decision based on a statistically correct interpretation of your models.

**ANSWER**

As mentioned above, I would not choose the MLE in this case since we do not have a significant amount of data and the MLE tends to overfit under scarcity of data. The Bayesian modeling process seems more appealing in this context since we want to have a sense of the uncertainty around the proportion of ratings, given the low number of observations. Also, by choosing the Bayesian modeling process, we are able to encode our realistic prior belief that most of the ratings are polarized (most users would give a rating either if a product is really good or really bad) and thus get a more realistic view of the posterior distribution of $\theta$. 

Having this in mind, if I had to choose a point estimate, this would be MAP or the posterior mean since it is based on a distribution which is more realistic given our assumptions (i.e. we have a reasonable prior belief which is updated after observing the data). 

Generally, it is easier to pick the optimal point estimate if you have a well defined loss function (for example, posterior mean is the optimal point estimate in case of squared loss function). However, in this setting we have not defined such a loss. MAP is the most likely parameter setting given the data, while the posterior mean is the posterior expected value of the parameter. Since the amount of data that we have is not large enough (especially for LW product), the posterior distribution would not be very peaked at the MAP. For this reasons, in this case I would choose the posterior mean estimate since it would be an average of all possible parameter settings weighted by how probable they are. Since, we don't have enough data to be very certain about the point estimate, in this way we would be able to propagate some of the uncertainty in our point estimate by averaging out instead of choosing the most probable from a wide distribution. 